In [33]:
# 本文所用评价指标参考博客：
# https://blog.csdn.net/sinat_26917383/article/details/70577710

In [34]:
import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics 

In [35]:
# http://cvc.cs.yale.edu/cvc/projects/yalefaces/yalefaces.html
# 由耶鲁大学计算视觉与控制中心创建，包含15位志愿者的165张图片，包含光照、表情和姿态的变化。
# Yale人脸数据库中一个采集志愿者的10张样本，
# 相比较ORL人脸数据库Yale库中每个对象采集的样本包含更明显的光照、表情和姿态以及遮挡变化。

In [36]:
os.getcwd()

'C:\\Users\\Tianh\\Desktop\\fth'

In [37]:
ls

 驱动器 C 中的卷是 OS
 卷的序列号是 7A99-DFFA

 C:\Users\Tianh\Desktop\fth 的目录

2019/10/19  09:49    <DIR>          .
2019/10/19  09:49    <DIR>          ..
2019/10/19  09:49    <DIR>          .ipynb_checkpoints
2019/10/19  09:47            18,936 data_experiment_ORL_data.ipynb
2019/10/19  09:48            18,675 data_experiment_WINE_white_Data.ipynb
2019/10/19  09:47            54,806 data_experiment_WINR__data.ipynb
2019/10/19  09:49            18,936 data_experiment_Yale_Data.ipynb
2019/10/18  17:50            13,071 Untitled.ipynb
2019/10/18  18:23            49,825 Untitled1.ipynb
2019/10/19  09:49               586 Untitled2.ipynb
               7 个文件        174,835 字节
               3 个目录 299,524,734,976 可用字节


In [38]:
#导入数据

In [39]:
data = scipy.io.loadmat('C:\\Users\\Tianh\\Desktop\\DMLab\\data\\face\\Yale_32x32.mat')

In [40]:
type(data)
data
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'fea', 'gnd'])

In [41]:
#对数据进行预处理

In [42]:
fea = data['fea']

In [43]:
fea

array([[ 24,  24,  26, ...,  73, 138, 205],
       [ 70,  68,  49, ..., 248, 255, 254],
       [ 74,  71,  64, ..., 254, 253, 164],
       ...,
       [ 59, 117,  58, ..., 200, 237, 241],
       [ 52,  39,  37, ..., 255, 245, 219],
       [ 15,  12,  11, ...,  49,  33,  16]], dtype=uint8)

In [44]:
print(fea.shape)
print(type(fea))
print(fea.dtype)
fea = data['fea'].astype(np.float64)
print(fea.dtype)
print(fea)

(165, 1024)
<class 'numpy.ndarray'>
uint8
float64
[[ 24.  24.  26. ...  73. 138. 205.]
 [ 70.  68.  49. ... 248. 255. 254.]
 [ 74.  71.  64. ... 254. 253. 164.]
 ...
 [ 59. 117.  58. ... 200. 237. 241.]
 [ 52.  39.  37. ... 255. 245. 219.]
 [ 15.  12.  11. ...  49.  33.  16.]]


In [45]:
# print(data['gnd'])
print(type(data['gnd']))
print(data['gnd'].shape)
true_label= data['gnd'].ravel()  #降维 返回视图
# print(gnd)
print(type(true_label))
print(true_label.shape)
print(true_label)

<class 'numpy.ndarray'>
(165, 1)
<class 'numpy.ndarray'>
(165,)
[ 1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  3  3
  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5
  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7
  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9
  9  9  9 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11
 11 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 14
 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 15]


In [46]:
#对数据进行归一化

In [47]:
#最大最小值归一化
# 参考博客：https://blog.csdn.net/zlzl8885/article/details/78094366
def minmax_Norm(dataSet):
    minVals = dataSet.min(0) # 取每一列的最小值
    maxVals = dataSet.max(0) # 取每一列的最大值
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    normDataSet = normDataSet/np.tile(ranges, (m, 1))   
    return normDataSet

In [48]:
# Z-score归一化
# 参考博客：https://blog.csdn.net/sinat_36458870/article/details/79498302#2Zscore_22
def ZScore_Norm(dataSet):  
    mu = dataSet.average()
    sigma = dataSet.std()
    x = (x - mu) / sigma
    return datsSet  

In [49]:
# Sigmoid归一化
# Sigmoid函数是一个具有S形曲线的函数，
# 是良好的阈值函数，在(0, 0.5)处中心对称，
# 在(0, 0.5)附近有比较大的斜率而当数据趋向于正无穷和负无穷的时候，
# 映射出来的值就会无限趋向于1和0
def sigmoid_Norm(dataSet,useStatus):  
    if useStatus:  
        return 1.0 / (1 + np.exp(-float(dataSet)))
    else:  
        return float(dataSet)
# useStatus管理是否使用sigmoid的状态，方便调试使用

In [50]:
norm_fea = minmax_Norm(fea)
norm_fea

array([[0.09411765, 0.09411765, 0.10196078, ..., 0.28627451, 0.54117647,
        0.80314961],
       [0.2745098 , 0.26666667, 0.19215686, ..., 0.97254902, 1.        ,
        0.99606299],
       [0.29019608, 0.27843137, 0.25098039, ..., 0.99607843, 0.99215686,
        0.64173228],
       ...,
       [0.23137255, 0.45882353, 0.22745098, ..., 0.78431373, 0.92941176,
        0.94488189],
       [0.20392157, 0.15294118, 0.14509804, ..., 1.        , 0.96078431,
        0.85826772],
       [0.05882353, 0.04705882, 0.04313725, ..., 0.19215686, 0.12941176,
        0.05905512]])

In [51]:
#k-means 聚类

In [58]:
cluster_num = 15

In [59]:
kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(fea)

In [60]:
predict_label  =kmeans.labels_

In [61]:
kmeans.cluster_centers_

array([[ 36.6       ,  40.9       ,  44.9       , ...,  53.7       ,
         55.        ,  62.5       ],
       [ 20.875     ,  27.75      ,  28.875     , ..., 228.875     ,
        213.375     , 181.125     ],
       [ 80.        ,  45.3       ,  34.5       , ...,  65.7       ,
         66.9       ,  80.7       ],
       ...,
       [ 82.        ,  67.45454545,  63.        , ..., 181.18181818,
        159.81818182, 140.36363636],
       [ 37.        ,  50.25      ,  68.        , ..., 254.75      ,
        240.75      , 244.5       ],
       [ 22.18181818,  22.45454545,  23.09090909, ..., 210.09090909,
        172.63636364, 153.54545455]])

In [1]:
#k-mediod 聚类

In [2]:
def pearson_distance(vector1, vector2):
    from scipy.spatial.distance import pdist
    X = vstack([vector1, vector2])
    d2 = pdist(X)
    return d2

In [3]:
distances_cache = {}

In [4]:
def totalcost(blogwords, costf, medoids_idx):
    size = len(blogwords)
    total_cost = 0.0
    medoids = {}
    for idx in medoids_idx:
        medoids[idx] = []
    for i in range(size):
        choice = None
        min_cost = inf
        for m in medoids:
            tmp = distances_cache.get((m, i), None)
            if tmp == None:
                tmp = pearson_distance(blogwords[m], blogwords[i])
                distances_cache[(m, i)] = tmp
            if tmp < min_cost:
                choice = m
                min_cost = tmp
        medoids[choice].append(i)
        total_cost += min_cost
    return total_cost, medoids

In [ ]:
def kmedoids(blogwords, k):
    import random
    size = len(blogwords)
    medoids_idx = random.sample([i for i in range(size)], k)
    pre_cost, medoids = totalcost(blogwords, pearson_distance, medoids_idx)
#     print pre_cost
    current_cost = inf  # maxmum of pearson_distances is 2.
    best_choice = []
    best_res = {}
    iter_count = 0
    while 1:
        for m in medoids:
            for item in medoids[m]:
                if item != m:
                    idx = medoids_idx.index(m)
                    swap_temp = medoids_idx[idx]
                    medoids_idx[idx] = item
                    tmp, medoids_ = totalcost(blogwords, pearson_distance, medoids_idx)
                    # print tmp,'-------->',medoids_.keys()
                    if tmp < current_cost:
                        best_choice = list(medoids_idx)
                        best_res = dict(medoids_)
                        current_cost = tmp
                    medoids_idx[idx] = swap_temp
        iter_count += 1
#         print current_cost, iter_count
        if best_choice == medoids_idx: break
        if current_cost <= pre_cost:
            pre_cost = current_cost
            medoids = best_res
            medoids_idx = best_choice

    return current_cost, best_choice, best_res

In [62]:
#聚类评价

In [63]:
#Mutual Information based scores 互信息
print(metrics.adjusted_mutual_info_score(true_label,predict_label))

# Homogeneity 同质性 每个群集只包含单个类的成员
print(metrics.homogeneity_score(true_label, predict_label))

# completeness 完整性 类的所有成员都分配给同一个群集
print(metrics.completeness_score(true_label, predict_label))

# 两者的调和平均V-measure：
print(metrics.v_measure_score(true_label, predict_label))

#  Fowlkes-Mallows scores
# The Fowlkes-Mallows score FMI is defined as the geometric mean 
# of the pairwise precision and recall:
print(metrics.fowlkes_mallows_score(true_label,predict_label))

# Silhouette Coefficient 轮廓系数
print(metrics.silhouette_score(fea, predict_label, metric='euclidean'))

# Calinski-Harabaz Index 分数值ss越大则聚类效果越好
# 类别内部数据的协方差越小越好，
# 类别之间的协方差越大越好，这样的Calinski-Harabasz分数会高。
print(metrics.calinski_harabaz_score(fea, predict_label))

0.3038688520384002
0.46837384203190396
0.5073769627444457
0.4870958778323812
0.2720501361715617
0.12284307028703369
11.411153255059657


D:\Users\Tianh\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\Users\Tianh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function calinski_harabaz_score is deprecated; Function 'calinski_harabaz_score' has been renamed to 'calinski_harabasz_score' and will be removed in version 0.23.
  warnings.warn(msg, category=DeprecationWarning)


In [30]:
#一些评价指标的实现代码

In [31]:
# 交叉熵及纯度实现参考博客：
# https://blog.csdn.net/vernice/article/details/46467449

In [ ]:
#老师上课所讲

In [6]:
# scipy.io.savemat()